In [1]:
import pandas as pd
import numpy as np
import datetime
import calendar
import math
import os
import xarray as xr
import rioxarray
import glob


input_folder = r"H:\MJI\MSP\Data\Downloaded\Chlorophyll\NASA\L3\Daily_Data"
output_path = r'H:\MJI\MSP\Data\Downloaded\Chlorophyll\NASA\L3\Monthly_from_Daily'

monthly_ds_list = []

for month_folder_path in glob.glob(os.path.join(input_folder, "**")):
    
    month_folder = month_folder_path.split(os.sep)[-1]
    list_count = 1
    daily_ds_list = []

    month_time = pd.to_datetime('20220101', format = "%Y%m%d")

    for file in glob.glob(r"{}\{}\*.nc4".format(input_folder, month_folder)):

        file_path_list = file.split(os.sep)
        date_text = file_path_list[-1].split('.')[1]

        ds = xr.open_dataset(file)

        aoi_dataset = ds.sel(lat=slice(25,15), lon=slice(80, 95))

        time = pd.to_datetime([date_text], format = "%Y%m%d")
        chlor_a = aoi_dataset['chlor_a']

        reshape_data = chlor_a.to_numpy().reshape(len(time), len(aoi_dataset['lat'].to_numpy()),
                                                  len(aoi_dataset['lon'].to_numpy()))

        td_dataset_name = 'td_dataset{}'.format(list_count)

        td_dataset_name = xr.Dataset(data_vars=dict(chlor_a =(["time", 'lat', 'lon'], reshape_data)),
                                     coords=dict(time = np.array(time),
                                                 lat = aoi_dataset['lat'].to_numpy(),
                                                 lon = aoi_dataset['lon'].to_numpy()),
                                     attrs = ds.attrs)

        daily_ds_list.append(td_dataset_name)
        list_count = list_count + 1
        month_time =  time

    daily_ds_comb = xr.concat(daily_ds_list , dim = 'time')

    monthly_mean_ds = 'mean_ds_{}'.format(month_time.month[0])
    monthly_mean_ds = daily_ds_comb.mean('time', keep_attrs=True, skipna=True)

    for var in monthly_mean_ds.data_vars:
            monthly_mean_ds[var] = monthly_mean_ds[var].expand_dims(time_monthly = month_time, axis=0)

    monthly_ds_list.append(monthly_mean_ds)
    
monthly_ds_comb = xr.concat(monthly_ds_list , dim = 'time_monthly')
monthly_ds_comb.to_netcdf(os.path.join(output_path, 'NASA_Mean_Monthly_Chloropyhll_Combined.nc'))

In [3]:
import xarray as xr
import rioxarray
import numpy as np
import netCDF4
import os

input_file_path = r"H:\MJI\MSP\Data\Downloaded\Chlorophyll\NASA\L3\Monthly_from_Daily\NASA_Mean_Monthly_Chloropyhll_Combined.nc"
ouput_folder_path = r"G:\MJI\SKZ\Script\MSP_Data\Output\Seasonal\Chlorophyll_NASA"

monthly_ds = xr.open_dataset(input_file_path, engine = 'netcdf4', decode_times= True, mask_and_scale = False)

winter_months = [time.to_numpy() for time in monthly_ds.time_monthly if time.dt.month in [1, 2, 12]]
summer_months = [time.to_numpy() for time in monthly_ds.time_monthly if time.dt.month in [6, 7, 8]]

season_dict = {'Winter': winter_months, 'Summer': summer_months}

season_ds_list = []

for season in season_dict.keys():
    
    season_ds = monthly_ds.sel(time_monthly = season_dict[season], lat = slice(24, 16), lon = slice(81, 95))
    season_mean_ds = season_ds.mean(dim= 'time_monthly', skipna=True, keep_attrs=True)

    season_array = np.array([season], dtype = str)
    
    for var in season_mean_ds.data_vars:
        season_mean_ds[var] = season_mean_ds[var].expand_dims(season=season_array, axis=0)
        
    ds_name = '{}_ds'.format(season)
    ds_name = season_mean_ds
    season_ds_list.append(ds_name)

combined_sesason_ds = xr.concat(season_ds_list , dim = 'season')
season_name_array = combined_sesason_ds['season'].to_numpy()

for i, season_name in zip(range(0, len(combined_sesason_ds['season'])), season_name_array):
    
    season_chloro_ds = combined_sesason_ds['chlor_a'][i,:,:]
    
    season_chloro_ds.rio.write_crs("epsg:4326", inplace=True)
    season_chloro_ds.rio.write_nodata(-9999, encoded=True, inplace=True)
    season_chloro_ds.rio.set_spatial_dims("lon", "lat", inplace=True)

    season_chloro_ds.rio.to_raster(os.path.join(ouput_folder_path, 'Chlorophyll_a_{}_2022.tif'.format(season_name)))
    
combined_sesason_ds.to_netcdf(os.path.join(ouput_folder_path, 'NASA_Seasonal_Chlorophyll.nc'))